In [1]:
# lettura di tutti i documenti da una cartella
from langchain_community.document_loaders import DirectoryLoader  # pip install unstructured

loader = DirectoryLoader('docs/', glob="**/*.txt")
documents = loader.load()

for doc in documents:
    print(doc, "\n\n")

libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.


page_content='Auto o due ruote… è da settimane che mi gira in testa questa domanda. L’auto ha una logica: comoda, versatile, climatizzatore d’estate e aria calda d’inverno. Puoi buttarci dentro la spesa, gli amici, perfino i bagagli. Ma ogni volta che mi immagino al volante vedo me stesso bloccato nel traffico, con lo sguardo fisso sul paraurti davanti, le mani che stringono il volante come se stessi aspettando che succeda qualcosa… e non succede mai. È come vivere metà della vita in una scatola di metallo ferma, parcheggiata o imbottigliata. No, l’auto non è libertà, è routine.

La moto invece… due ruote sono vento, scia, movimento. È il cuore che accelera quando apri il gas, è la città che si apre come un labirinto da attraversare veloce, fluido, senza catene. È libertà che senti sulla pelle, nelle braccia, persino nel rumore cupo del motore che vibra sotto di te. Non c’è paragone: voglio una moto.

Però… scooter o moto? Lo scooter è furbo, certo: pratico, ti porta al lavoro senza pe

In [2]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage._lc_store import create_kv_docstore  #  <---
from langchain.storage import LocalFileStore


In [3]:
# Text-Splitter per i chunk di documento più grandi, da usare come dati recuperati
parent_splitter = RecursiveCharacterTextSplitter(chunk_size=350, chunk_overlap=150)

# Secondo Text-Splitter, per la creazione di chunk più piccoli su cui indicizzare le ricerche
child_splitter = RecursiveCharacterTextSplitter(chunk_size=85, chunk_overlap=15)

vectorstore = Chroma(collection_name="split_parents",
                     embedding_function=OpenAIEmbeddings())

# dizionario su filesystem utilizzato per recuperare i documenti più grandi, una volta
# identificati tramite ricerca sui chunk più piccoli
fs = LocalFileStore("./storage_locale")
store = create_kv_docstore(fs)

In [4]:
# ParentDocumentRetriever gestisce automaticamente
# il "doppio livello" di granularità dei chunk

retriever = ParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

retriever.add_documents(documents)

In [5]:
# dai 2 documenti di partenza, vengono creati 
# 23 chunk da 350 caratteri massimo
# memorizzati in `store`, mentre i chunk più
# piccoli, da massimo 85 caratteri, sono
# memorizzati nel vector-store
len(list(store.yield_keys()))

23

In [6]:
store.store

In [7]:
store.store.root_path

WindowsPath('c:/Users/vinor/Desktop/Develhope/projects/EDU-RAG/storage_locale')

In [8]:
# accedendo al VectorStore (Chroma), tramite similarity search, la ricerca viene effettuata
# sui chunk estratti più piccoli
sub_docs = vectorstore.similarity_search("in quale dialogo si parla di prezzi?")

sub_docs[0]

Document(id='873d644a-d742-4c2d-80a2-215222da88c5', metadata={'source': 'docs\\viaggio.txt', 'doc_id': '4952d6d0-574b-44da-9110-635654f1c65e'}, page_content='muoversi subito, perché i prezzi salgono.')

In [9]:
for s in sub_docs:
    print(s)

page_content='muoversi subito, perché i prezzi salgono.' metadata={'source': 'docs\\viaggio.txt', 'doc_id': '4952d6d0-574b-44da-9110-635654f1c65e'}
page_content='Lui: Esatto. Da Roma o Milano si trovano biglietti intorno ai 900-1100 euro a testa,' metadata={'source': 'docs\\viaggio.txt', 'doc_id': '4952d6d0-574b-44da-9110-635654f1c65e'}
page_content='evitare. Vale la pena leggerlo bene.' metadata={'doc_id': '7bc4ac12-d6cb-4299-a8e0-a6ecf1984359', 'source': 'docs\\viaggio.txt'}
page_content='Piuttosto: valigia o zaino?' metadata={'doc_id': '9fa016e6-fcdf-4a23-ad05-67a5b4e4bb31', 'source': 'docs\\viaggio.txt'}


In [10]:
# il retriever ritorna invece il chunk di documento più grande
retrieved_docs = retriever.invoke("in quale dialogo si parla di prezzi?")

retrieved_docs[0]

Document(metadata={'source': 'docs\\viaggio.txt'}, page_content='Lui: Esatto. Da Roma o Milano si trovano biglietti intorno ai 900-1100 euro a testa, se prenotiamo presto. Magari con Iberia o Latam. Conviene muoversi subito, perché i prezzi salgono.')

In [11]:
for r in retrieved_docs:
    print(r)

page_content='Lui: Esatto. Da Roma o Milano si trovano biglietti intorno ai 900-1100 euro a testa, se prenotiamo presto. Magari con Iberia o Latam. Conviene muoversi subito, perché i prezzi salgono.' metadata={'source': 'docs\\viaggio.txt'}
page_content='Lei: Giusto. Intanto, per la sicurezza ho controllato che sul sito Viaggiare Sicuri del Ministero danno consigli su vaccini, altitudine e zone da evitare. Vale la pena leggerlo bene.

Lui: Sì, meglio. Per esempio so che in alcune zone dicono di fare attenzione ai trasporti notturni. E ci ricordano pure di avere sempre un’assicurazione sanitaria.' metadata={'source': 'docs\\viaggio.txt'}
page_content='Lei: A proposito, che tipo di abbigliamento portiamo? Io pensavo a strati: magliette leggere per il giorno, felpe e giacche a vento per la sera. A Cusco di notte fa freddo anche d’estate.

Lui: Concordo. Scarpe comode da trekking, cappello e crema solare. Piuttosto: valigia o zaino?' metadata={'source': 'docs\\viaggio.txt'}
